In [1]:
from torch_geometric.datasets import Planetoid
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

In [17]:
x = torch.tensor([[-1, 0, 1], [0, 1, 2]], dtype=torch.float)
y = torch.tensor([0, 1,3,4,5,6,], dtype=torch.long)

In [20]:
x.flatten()

tensor([-1.,  0.,  1.,  0.,  1.,  2.])

In [5]:
data = Data(x=x, edge_index=edge_index, y=y)
data.num_nodes

2

In [3]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [4]:
custom_dataset = CustomDataset(dataset_list)
custom_dataset[0]

Data(x=[2, 3], edge_index=[2, 2], y=[2])

In [5]:
dataloader = DataLoader(custom_dataset, batch_size=1, shuffle=True)

/home/btr/miniconda3/envs/LLMEnG/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [6]:
for batch in dataloader:
    print(batch.x)
    print(batch.y)

tensor([[-1.,  0.,  1.],
        [ 0.,  1.,  2.]])
tensor([0, 1])


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(3, 3, aggr='mean')
        self.conv2 = SAGEConv(3, 2, aggr='mean')

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        print(x)
        # x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return x

In [14]:
model = Net()
optizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

model.train()
for epoch in range(10):
    for data in dataloader:
        optizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        unique_batch_indices = torch.unique(data.batch)
        print(unique_batch_indices)
        # 迭代并处理每个图
        loss.backward()
        optizer.step()

tensor([[ 0.6433,  0.2546,  0.2448],
        [ 0.4562,  1.4739, -0.6014]], grad_fn=<AddBackward0>)
tensor([0])
Data(x=[2, 3], edge_index=[2, 2], y=[2])
子图 0 的节点特征：
tensor([[-1.,  0.,  1.],
        [ 0.,  1.,  2.]])

tensor([[ 0.6372,  0.2498,  0.2353],
        [ 0.4631,  1.4760, -0.5987]], grad_fn=<AddBackward0>)
tensor([0])
Data(x=[2, 3], edge_index=[2, 2], y=[2])
子图 0 的节点特征：
tensor([[-1.,  0.,  1.],
        [ 0.,  1.,  2.]])

tensor([[ 0.6311,  0.2450,  0.2260],
        [ 0.4699,  1.4782, -0.5960]], grad_fn=<AddBackward0>)
tensor([0])
Data(x=[2, 3], edge_index=[2, 2], y=[2])
子图 0 的节点特征：
tensor([[-1.,  0.,  1.],
        [ 0.,  1.,  2.]])

tensor([[ 0.6251,  0.2402,  0.2169],
        [ 0.4766,  1.4806, -0.5934]], grad_fn=<AddBackward0>)
tensor([0])
Data(x=[2, 3], edge_index=[2, 2], y=[2])
子图 0 的节点特征：
tensor([[-1.,  0.,  1.],
        [ 0.,  1.,  2.]])

tensor([[ 0.6192,  0.2353,  0.2080],
        [ 0.4833,  1.4830, -0.5909]], grad_fn=<AddBackward0>)
tensor([0])
Data(x=[2, 3], edge_index

In [1]:
a = [1,2,3]
b = [4,5,6]
a.extend(b)

21